# Understanding redrock output

This tutorial describes how to combine the zbest fit coefficients with the
redrock templates to see how the best fit template compares to the actual data.

See https://desi.lbl.gov/trac/wiki/Computing/JupyterAtNERSC to configure Jupyter at NERSC to be able to run this
tutorial using jupyter.nersc.gov and the "19.12" data in `/global/cfs/cdirs/desi/datachallenge/reference_runs/19.12`.

If running locally, see https://desi.lbl.gov/trac/wiki/Pipeline/GettingStarted/Laptop for installing code and download the the 19.12 reference run data set from NERSC at `/global/cfs/cdirs/desi/datachallenge/reference_runs/19.12`.

The bottom of this tutorial lists the code versions used while testing this.

Stephen Bailey<br/>
January 2020

## Basic setup

In [ ]:
import os
from astropy.table import Table
import numpy as np
import desispec.io
# import redrock.io
%pylab inline

Add the tutorials directory to the python search path so that we can import desitutorials

In [ ]:
sys.path.append(os.path.abspath(os.getcwd()+'/..'))
import desitutorials

For convenience, move into the directory that contains the reference run data and set some environment variables so that desispec can find the files.  If you are running locally (*e.g.* on your laptop instead of at jupyter.nersc.gov, replace these with wherever you installed the reference run data.

In [ ]:
os.chdir('/global/cfs/cdirs/desi/datachallenge/reference_runs/19.12')
os.environ['DESI_SPECTRO_REDUX'] = os.path.join(os.getcwd(), 'spectro', 'redux')
os.environ['SPECPROD'] = 'mini'

## Read in some spectra and the matching zbest file

In [ ]:
specfile = desispec.io.findfile('spectra', groupname='5299', nside=64)
zbestfile = desispec.io.findfile('zbest', groupname='5299', nside=64)

spectra = desispec.io.read_spectra(specfile)
zbest = Table.read(zbestfile, hdu=1)

What did we get?

In [ ]:
zbest['SPECTYPE', 'Z'][0:5]

In [ ]:
for spectype in set(zbest['SPECTYPE']):
    ii = (zbest['SPECTYPE'] == spectype)
    hist(zbest['Z'][ii], 41, (-0.1,4), label=spectype.strip())
legend()

Note that redrock does not distinguish between ELGs and LRGs: both are just `spectype=='GALAXY'`.

## Compare redrock templates to spectra

`redrock.io.read_templates` returns a list of templates, then convert those into a dict of template type -> Template object.  Ignore INFO messages about using default values for redshift ranges.

In [ ]:
# tx = redrock.io.read_templates()
import redrock.templates

templates = dict()
for filename in redrock.templates.find_templates():
    t = redrock.templates.Template(filename)
    templates[(t.template_type, t.sub_type)] = t

Pick a target and use its zbest coefficients to construct the redrock template fit for that
object.

In [ ]:
i = 1
z = zbest['Z'][i]
targetid = zbest['TARGETID'][i]
spectype = zbest['SPECTYPE'][i].strip()
subtype = zbest['SUBTYPE'][i].strip()
fulltype = (spectype, subtype)
ncoeff = templates[fulltype].flux.shape[0]
coeff = zbest['COEFF'][i][0:ncoeff]

In [ ]:
tflux = templates[fulltype].flux.T.dot(coeff)
twave = templates[fulltype].wave * (1+z)

In [ ]:
ispec = np.where(spectra.fibermap['TARGETID'] == targetid)[0][0]

figure(figsize=(8,5))
subplot(211)
maxflux = 0.0
for band in spectra.bands:
    plot(spectra.wave[band], spectra.flux[band][ispec], 'k-', alpha=0.5)
    maxflux = max(maxflux, np.max(spectra.flux[band][ispec]))

plot(twave, tflux, 'r-')
xlim(3500, 10000)
ylim(-1, 0.9*maxflux)

#- zoom in on [OII]
subplot(212)
for band in spectra.bands:
    plot(spectra.wave[band], spectra.flux[band][ispec], 'k-', alpha=0.5)

plot(twave, tflux, 'r-')
xlim(3727*(1+z)-100, 3727*(1+z)+100)
ylim(-1,2)

That wasn't quite correct because the redrock templates are high resolution
before any instrumental resolution is applied.  Let's update that to include
the spectral resolution.

In [ ]:
from desispec.interpolation import resample_flux
from desispec.resolution import Resolution
R = Resolution(spectra.resolution_data['r'][i])
txflux = R.dot(resample_flux(spectra.wave['r'], twave, tflux))

In [ ]:
plot(spectra.wave['r'], spectra.flux['r'][ispec])
plot(spectra.wave['r'], txflux)
# ylim(-1,2)
xlim(3727*(1+z)-100, 3727*(1+z)+100)

## Future work

**TODO**: document redrock rr*.h5 files, e.g. the chi2 vs. redshift and how to get the second, third, etc. best fits

## Code versions

In [ ]:
print('Code versions used for this tutorial:')
desitutorials.print_code_versions()